In [1]:
import pandas as pd
import os
import random
import hashlib
import datetime

In [2]:
parquet_path = os.path.join(os.getcwd(), "file.parquet")
parquet_group_path = os.path.join(os.getcwd(), "group.parquet")

In [3]:
!pip install pyarrow

In [4]:
pd.read_parquet(parquet_path, engine='auto')

,Brand,Model,OS,SSD,Price,Type_Of_Gadget,Discripton
0,Samsung,5,OS,225gb,913.471618,None,None
1,Nokia,8,OS,225gb,1008.034365,None,None
2,Apple,3,OS,225gb,793.393242,None,None
3,Apple,2,OS,225gb,1057.636751,None,None
4,Nokia,8,OS,225gb,975.501182,None,None
...,...,...,...,...,...,...,...
115,DELL,7,None,None,1028.226686,PC,Lorem ipsum
116,Nokia,8,None,None,897.237303,Telephon,Lorem ipsum
117,Apple,1,None,None,943.040868,Telephon,Lorem ipsum
118,Nokia,8,None,None,1018.369196,Telephon,Lorem ipsum


1.)EXTRACT

In [5]:
limit = 10

In [6]:
from config import TYPE_OF_GADGET, BRAND, MODEL, DISCRIPTION, PRICE, OS, SSD, TELEPHON, PC

In [7]:
jsonAttributesFrom1stApi = [TYPE_OF_GADGET, BRAND, MODEL, DISCRIPTION, PRICE] #data from this API has different type of gadgets

In [8]:
jsonAttributesFrom2ndApi = [BRAND, MODEL, OS, SSD, PRICE] #data from this API has only telephone

In [9]:
gadgetsType = [TELEPHON, PC]

telephonBrand = ["Apple", "Samsung", "Nokia"]
pcBrand = ["HP", "DELL", "Apple"]

BrandsModels = {}

BrandsModels[TELEPHON] = {
    "Apple": ["1", "2", "3"],
    "Samsung": ["4", "5"],
    "Nokia":["6", "7", "8"]
}
BrandsModels[PC] = {
    "HP": ["1", "a"],
    "DELL": ["7"],
    "Apple": ["b", "c"]
}

In [10]:
commonColumns = jsonAttributesFrom1stApi + jsonAttributesFrom2ndApi

commonColumns = list(dict.fromkeys(commonColumns))

commonColumns

['Type_Of_Gadget', 'Brand', 'Model', 'Discripton', 'Price', 'OS', 'SSD']

In [11]:
def gereneratePrice():
    return 1000 + random.triangular(-250, 250, 0)

In [12]:
df1 = []
for i in range(limit):
    gadget = {}
    gadget[TYPE_OF_GADGET] = random.choice(gadgetsType)
    gadget[BRAND] = random.choice(telephonBrand) if gadget[TYPE_OF_GADGET] == TELEPHON else random.choice(pcBrand)
    gadget[MODEL] = random.choice(BrandsModels[gadget[TYPE_OF_GADGET]][gadget[BRAND]])
    gadget[DISCRIPTION] = 'Lorem ipsum'
    gadget[PRICE] = gereneratePrice()
    df1.append(gadget)

df1 = pd.DataFrame(df1)
df1

,Type_Of_Gadget,Brand,Model,Discripton,Price
0,PC,DELL,7,Lorem ipsum,864.005420
1,PC,DELL,7,Lorem ipsum,886.427077
2,PC,HP,1,Lorem ipsum,977.352950
3,Telephon,Apple,3,Lorem ipsum,863.512293
4,Telephon,Samsung,5,Lorem ipsum,1151.579381
5,Telephon,Samsung,5,Lorem ipsum,1115.399528
6,PC,HP,a,Lorem ipsum,991.584055
7,Telephon,Apple,2,Lorem ipsum,1165.543106
8,Telephon,Nokia,7,Lorem ipsum,924.559902
9,Telephon,Apple,3,Lorem ipsum,1059.468728


In [13]:
df2 = []
for i in range(limit):
    gadget = {}
    gadget[BRAND] = random.choice(telephonBrand)
    gadget[MODEL] = random.choice(BrandsModels[TELEPHON][gadget[BRAND]])
    gadget[OS] = 'OS'
    gadget[SSD] = '225gb'
    gadget[PRICE] = gereneratePrice()
    df2.append(gadget)

df2 = pd.DataFrame(df2)
df2

,Brand,Model,OS,SSD,Price
0,Apple,1,OS,225gb,1014.352128
1,Nokia,6,OS,225gb,1116.718719
2,Apple,1,OS,225gb,872.388668
3,Apple,1,OS,225gb,920.357404
4,Samsung,5,OS,225gb,969.232517
5,Samsung,4,OS,225gb,1090.533971
6,Nokia,6,OS,225gb,913.028712
7,Apple,1,OS,225gb,952.129256
8,Nokia,8,OS,225gb,917.764902
9,Apple,3,OS,225gb,994.517163


2.)LOAD

In [14]:
df3 = df2.append(df1, ignore_index=True)

df3

,Brand,Model,OS,SSD,Price,Type_Of_Gadget,Discripton
0,Apple,1,OS,225gb,1014.352128,NaN,NaN
1,Nokia,6,OS,225gb,1116.718719,NaN,NaN
2,Apple,1,OS,225gb,872.388668,NaN,NaN
3,Apple,1,OS,225gb,920.357404,NaN,NaN
4,Samsung,5,OS,225gb,969.232517,NaN,NaN
5,Samsung,4,OS,225gb,1090.533971,NaN,NaN
6,Nokia,6,OS,225gb,913.028712,NaN,NaN
7,Apple,1,OS,225gb,952.129256,NaN,NaN
8,Nokia,8,OS,225gb,917.764902,NaN,NaN
9,Apple,3,OS,225gb,994.517163,NaN,NaN


In [15]:
df4 = pd.read_parquet(parquet_path, engine='auto')

df4 = df4.append(df3, ignore_index=True)

df4

,Brand,Model,OS,SSD,Price,Type_Of_Gadget,Discripton
0,Samsung,5,OS,225gb,913.471618,None,None
1,Nokia,8,OS,225gb,1008.034365,None,None
2,Apple,3,OS,225gb,793.393242,None,None
3,Apple,2,OS,225gb,1057.636751,None,None
4,Nokia,8,OS,225gb,975.501182,None,None
...,...,...,...,...,...,...,...
135,Samsung,5,NaN,NaN,1115.399528,Telephon,Lorem ipsum
136,HP,a,NaN,NaN,991.584055,PC,Lorem ipsum
137,Apple,2,NaN,NaN,1165.543106,Telephon,Lorem ipsum
138,Nokia,7,NaN,NaN,924.559902,Telephon,Lorem ipsum


In [16]:
df4.to_parquet(parquet_path, compression='gzip')

3.)TRANSFORM

In [17]:
df5  = pd.read_parquet(parquet_path, engine='auto')

df5 = pd.DataFrame(df5)

df5 = df5.drop_duplicates()

df5.fillna(value="MISSING", inplace=True)

df5

,Brand,Model,OS,SSD,Price,Type_Of_Gadget,Discripton
0,Samsung,5,OS,225gb,913.471618,MISSING,MISSING
1,Nokia,8,OS,225gb,1008.034365,MISSING,MISSING
2,Apple,3,OS,225gb,793.393242,MISSING,MISSING
3,Apple,2,OS,225gb,1057.636751,MISSING,MISSING
4,Nokia,8,OS,225gb,975.501182,MISSING,MISSING
...,...,...,...,...,...,...,...
135,Samsung,5,MISSING,MISSING,1115.399528,Telephon,Lorem ipsum
136,HP,a,MISSING,MISSING,991.584055,PC,Lorem ipsum
137,Apple,2,MISSING,MISSING,1165.543106,Telephon,Lorem ipsum
138,Nokia,7,MISSING,MISSING,924.559902,Telephon,Lorem ipsum


4.)GROUP BY

In [18]:
df6 = df5.loc[df5[TYPE_OF_GADGET] == TELEPHON]

df6.to_parquet(parquet_group_path, engine='auto')

5.)MD5 and SHA-256

In [19]:
hasher1 = hashlib.md5()
hasher2 = hashlib.sha256()

with open(parquet_group_path, 'rb') as open_file:
    content = open_file.read()
    hasher1.update(content)
    hasher2.update(content)
md5 = hasher1.hexdigest()
sha = hasher2.hexdigest()
print("MD5 =", md5, "\n")
print("SHA-256 =", sha)

MD5 = ff5a7c1aee0ad7c7eba81704a6971805 

SHA-256 = 1638337425282dac73e2e619721433fe29226bc52914c1084b315e67d900e7a2


In [20]:
df_hash = pd.DataFrame({
    "date": [datetime.datetime.now()],
    "MD5": [md5],
    "SHA-256": [sha]
})

df_hash

,date,MD5,SHA-256
0,2022-06-05 19:13:53.595597,ff5a7c1aee0ad7c7eba81704a6971805,1638337425282dac73e2e619721433fe29226bc52914c1...


In [21]:
df_tail_hash = pd.read_csv('hash.table.csv')

df_tail_hash = df_tail_hash.tail(1)

df_tail_hash

,date,MD5,SHA-256
0,2022-06-05 19:09:57.701021,3e47e864735edec6afd3b3a4ae5ea28b,08f1ce9b7ca0b6c2a03855a59d9ee31f6594e136d83016...


In [22]:
condition = (df_hash == df_tail_hash).any(1).any()
print(f"Has the parquet been chnged ? {condition}")

Has the parquet been chnged ? False


In [23]:
if not condition:
    df_hash.to_csv("hash.table.csv", mode='a', index=False, header=False)

In [24]:
pd.read_csv('hash.table.csv')

,date,MD5,SHA-256
0,2022-06-05 19:09:57.701021,3e47e864735edec6afd3b3a4ae5ea28b,08f1ce9b7ca0b6c2a03855a59d9ee31f6594e136d83016...
1,2022-06-05 19:13:53.595597,ff5a7c1aee0ad7c7eba81704a6971805,1638337425282dac73e2e619721433fe29226bc52914c1...
